In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 61 kB 468 kB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig,AdamW , get_linear_schedule_with_warmup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import itertools
import pandas as pd

In [4]:
cd /content/drive/MyDrive/master_thesis/notebooks/

/content/drive/MyDrive/master_thesis/notebooks


In [5]:
ls

DNABERT_reg_CNN.ipynb  DNABERT_reg.ipynb  inputparser.ipynb


In [6]:
cd /content/drive/MyDrive/master_thesis/inputs/

/content/drive/MyDrive/master_thesis/inputs


In [7]:
final_matrix = pd.read_csv("hg38_msxTm_ENCFF910TAZ.csv")

In [8]:
cd /content/drive/MyDrive/master_thesis/models/6-new-12w-0/

/content/drive/MyDrive/master_thesis/models/6-new-12w-0


In [9]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

Using GPU.


In [10]:
DNABERT_Tokenizer = BertTokenizer(vocab_file = "vocab.txt", config = "tokenizer_config.json")

In [11]:
def seq2kmer(seq, k = 6):
    """
    Function provided by Ji et al. (https://github.com/jerryji1993/DNABERT)
    Will convert given sequence into kmer.
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmer

In [12]:
def tokenize(seq_kmers, labels):
  input_ids = []
  attention_masks = []

  for sent in seq_kmers:
      encoded_dict = DNABERT_Tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          # Pad & truncate all sentences. (does not apply for our data since the lengths are all the same)
                          max_length = 297,
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attention masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Combine the training inputs into a TensorDataset.
  dataset = TensorDataset(input_ids, attention_masks, labels)
  return dataset

In [13]:
seq_kmers = list(map(seq2kmer, final_matrix["Sequence"].tolist()))
dataset = tokenize(seq_kmers, final_matrix["logTPM"].tolist())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=1)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.25, random_state=1)

In [15]:
model_config_path = "/content/drive/MyDrive/master_thesis/models/6-new-12w-0/config.json"
model_config =BertConfig.from_pretrained(model_config_path, num_labels = 1)

In [61]:
model_weight_path = "/content/drive/MyDrive/master_thesis/models/6-new-12w-0/pytorch_model.bin"
#this load the pre-trained model WITHOUT the linear layer from BertForSequenceClassification
dnabert_CNN = BertForSequenceClassification.from_pretrained(model_weight_path, config = model_config).base_model

Some weights of the model checkpoint at /content/drive/MyDrive/master_thesis/models/6-new-12w-0/pytorch_model.bin were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized 

In [59]:
class CNN_classifier(nn.Module):
    def __init__(self, num_labels = 1):

        self.conv_block1 = nn.Sequential(
            nn.Conv1d(768, FILTERS, kernel_size=3,stride=1),
            nn.BatchNorm1d(FILTERS),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(FILTERS, FILTERS, kernel_size=3,stride=1),
            nn.BatchNorm1d(FILTERS),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.dense1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(FILTERS*10, FILTERS)
        )
        self.dense2 = nn.Sequential(
            nn.Linear(FILTERS, FILTERS)
        )
        self.output = nn.Linear(FILTERS, num_labels)

    def forward(self, outputs):

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        pooled_output = self.conv_block1(pooled_output.unsqueeze(1))
        pooled_output = self.conv_block2(pooled_output)
        pooled_output = self.conv_block2(pooled_output)

        pooled_output = self.dense1(pooled_output)
        pooled_output = self.dense2(pooled_output)
        
        logits = self.output(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs

In [62]:
dnabert_CNN.classifier = CNN_classifier

In [ ]:
vars(dnabert_CNN)

In [ ]:
batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
prediction_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
batch_lr = {8:3e-4, 16:1e-4, 32: 5e-5, 64: 3e-5, 128: 3e-5}

optimizer = AdamW(dnabert.parameters(), lr=batch_lr[batch_size])

In [ ]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
                                            

In [ ]:
model = dnabert
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [ ]:
def train(model, optimizer, scheduler, epochs,       
          train_dataloader, validation_dataloader, device, clip_value=2):
    training_stats = []

    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        total_train_loss = 0
        for batch in train_dataloader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            #https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
            
            optimizer.zero_grad()
            model.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            total_train_loss += loss.item()
            loss.backward()
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print("  Average training loss: {0:.2f}".format(avg_train_loss))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.
        
        print("")
        print("Running Validation...")

        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():
                output = model(b_input_ids, 
                                      token_type_ids=None, 
                                      attention_mask=b_input_mask,
                                      labels=b_labels)
            
            loss = output[0]
            # Accumulate the validation loss.
            total_eval_loss += loss.item()


        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
            }
        )
    return model, training_stats

In [ ]:
ft_dnabert_model, training_stats = train(model, optimizer, scheduler, epochs, 
              train_dataloader, validation_dataloader, device, clip_value=2)